# Worldbank country indicator analysis
in this notebook we will:
1. get raw data from excel
2. get isocode from api
3. get data from worldbank


to import excel we use pandas and create a dataframe with excel data

to fetch data using API we import requests

In [3]:
import pandas as pd
import requests

load the df variable in the previous work 

In [6]:
df = pd.read_pickle("../session1/savedDF.pkl")

In [7]:
countries = pd.read_pickle("../session1/savedCountries.pkl")

get isocode for each country

In [18]:
iso_codes = {}

In [22]:
for index, name in countries.items():    
    if name in iso_codes:
        continue        
    cleanedName = name.split(' (')[0].strip()
    if cleanedName == "Thailiand":
        cleanedName = "Thailand"
    if cleanedName == "The United Arab Emirates":
        cleanedName = "United Arab Emirates"
    url = 'http://www.restcountries.eu/rest/v2/name/'+cleanedName
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        country_code = data[0]['alpha3Code']
        iso_codes[name] = country_code
    else:
        print('[ERROR]['+ name +'] we cannot get the data, reason is ', r.status_code, r.text)

In [23]:
iso_codes

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Armenia': 'ARM',
 'Azerbaijan': 'AZE',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Belarus': 'BLR',
 'Bhutan': 'BTN',
 'Bosnia and Herzegovina': 'BIH',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Cambodia': 'KHM',
 'China': 'CHN',
 'Croatia': 'HRV',
 'Czech Republic': 'CZE',
 'East Timor': 'TLS',
 'Egypt': 'EGY',
 'Estonia': 'EST',
 'France (Not BRI)': 'FRA',
 'Georgia': 'GEO',
 'Germany (not BRI)': 'DEU',
 'Hungary': 'HUN',
 'India': 'IOT',
 'Indonesia': 'IDN',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Israel': 'ISR',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kuwait': 'KWT',
 'Kyrgyzstan': 'KGZ',
 'Laos': 'LAO',
 'Latvia': 'LVA',
 'Lebanon': 'LBN',
 'Lithuania': 'LTU',
 'Macedonia': 'MKD',
 'Malaysia': 'MYS',
 'Maldives': 'MDV',
 'Moldova': 'MDA',
 'Mongolia': 'MNG',
 'Montenegro': 'MNE',
 'Myanmar': 'MMR',
 'Nepal': 'NPL',
 'Oman': 'OMN',
 'Pakistan': 'PAK',
 'Philippines': 'PHL',
 'Poland': 'POL',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russia': 'RUS',
 

we add a new iso_codes column to the df

In [25]:
df['isocode']=countries.apply(lambda x: iso_codes[x])

In [26]:
df[['isocode','Country (according to CN Gov):']]

,isocode,Country (according to CN Gov):
1,AFG,Afghanistan
2,ALB,Albania
3,ARM,Armenia
4,AZE,Azerbaijan
5,BHR,Bahrain
6,BGD,Bangladesh
7,BLR,Belarus
8,BTN,Bhutan
9,BIH,Bosnia and Herzegovina
10,BRN,Brunei


get data from worldbank

In [28]:
indicators = {}

In [32]:
tab = {'IMF WB\nCustoms and other import duties (% of tax revenue) 2016': 'GC.TAX.IMPT.ZS', 'WB IMF Non-performing loans to total gross loans ratio (%)': 'FB.AST.NPER.ZS', 'UIC Rail freight (mn ton x km travelled)': 'IS.RRS.GOOD.MT.K6', 'ICAO air freight (mn ton x km travelled)': 'IS.AIR.GOOD.MT.K1', 'WB Household final consumption expenditure, PPP bn USD': 'NE.CON.PRVT.PP.CD', 'WB UNESCO R&D Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WB UNESCO Researchers p. mn inhabitants': 'SP.POP.SCIE.RD.P6', 'IFDI Net Inflows (% of GDP)': 'BX.KLT.DINV.WD.GD.ZS', 'WHO Current Health Expenditure p.c., PPP USD': 'SH.XPD.CHEX.PP.CD', 'WB Research Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WIPO Trademark applications': 'IP.TMK.RSCT',
'WIPO Patent Applications': 'IP.PAT.RESD'}

In [38]:
for column, indicatorCode in tab.items():
    if column not in indicators:
        indicators[column] = {}       
    for index, isocode in df['isocode'].items():    
        url = 'http://api.worldbank.org/v2/country/' + isocode + '/indicator/'+ indicatorCode +'?mrv=1&format=json'
        r = requests.get(url)
        if r.status_code == 200:
            #print(isocode)
            data = r.json()        
            try:
                indicators[column][isocode] = data[1][0]['value']
                #print(indicators[column][isocode])
            except:
                indicators[column][isocode] = "NA"
                #print(indicators[column][isocode])
        else:
            print('[ERROR]['+ isocode +'] we cannot get the data, reason is ', r.status_code, r.text)

save dataframe

In [ ]:
df.to_pickle("savedDF.pickle")

we insert the fetched data into the df

In [45]:
for column, indicatorCode in indicators.items():
    df[column] = df['isocode'].apply(lambda x: indicatorCode[x])

create new excel sheet with filled data

In [ ]:
df.to_excel('Data-sheet-STUDENTS-filled.xlsx')

In [48]:
df[['isocode','Country (according to CN Gov):','IMF WB\nCustoms and other import duties (% of tax revenue) 2016', 'WB IMF Non-performing loans to total gross loans ratio (%)', 'UIC Rail freight (mn ton x km travelled)', 'ICAO air freight (mn ton x km travelled)', 'WB Household final consumption expenditure, PPP bn USD', 'WB UNESCO R&D Expenditure % of GDP', 'WB UNESCO Researchers p. mn inhabitants', 'IFDI Net Inflows (% of GDP)', 'WHO Current Health Expenditure p.c., PPP USD', 'WB Research Expenditure % of GDP', 'WIPO Trademark applications','WIPO Patent Applications']]

,isocode,Country (according to CN Gov):,IMF WB Customs and other import duties (% of tax revenue) 2016,WB IMF Non-performing loans to total gross loans ratio (%),UIC Rail freight (mn ton x km travelled),ICAO air freight (mn ton x km travelled),"WB Household final consumption expenditure, PPP bn USD",WB UNESCO R&D Expenditure % of GDP,WB UNESCO Researchers p. mn inhabitants,IFDI Net Inflows (% of GDP),"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications
1,AFG,Afghanistan,35.2031,12.2018,NA,29.5593,5.21431e+10,NA,NA,0.718898,162.781,NA,NA,NA
2,ALB,Albania,2.21646,13.2325,46,0,2.44298e+10,0.15412,156.101,8.01551,759.667,0.15412,1232,14
3,ARM,Armenia,6.27539,5.43232,689.46,0,2.4646e+10,0.2277,NA,2.04411,876.857,0.2277,2685,107
4,AZE,Azerbaijan,7.57185,NA,4633,44.0886,1.01816e+11,0.18521,NA,2.98895,1193.06,0.18521,2817,144
5,BHR,Bahrain,27.3861,NA,NA,420.984,2.58919e+10,0.10116,368.902,4.01407,1866.3,0.10116,253,8
6,BGD,Bangladesh,28.923,8.39791,710,63.8182,4.68641e+11,NA,NA,1.07298,90.5984,NA,9247,61
7,BLR,Belarus,8.20062,12.8522,48538,1.9004,1.27064e+11,0.58716,NA,2.47207,1151.41,0.58716,4453,434
8,BTN,Bhutan,2.8405,8.41731,NA,0.690916,3.834e+09,NA,NA,0.104527,293.11,NA,16,4
9,BIH,Bosnia and Herzegovina,0.00015536,10.0463,1114.33,8.7e-05,3.14871e+10,0.19951,463.899,2.45426,1123.43,0.19951,742,87
10,BRN,Brunei,NA,3.5384,NA,129.352,4.72344e+09,0.03701,283.431,3.77013,1812.41,0.03701,176,8
